# Table of Content
1. [Text Cleaning](#textcleaning)
2. [Text Preprocessing](#textpreprocessing)
3. [EDA](#EDA)
4. [Feature Engineering](#featureengineering)

In [3]:
%time
import os
import sys
import time
import random
import string
from ast import literal_eval
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
import numpy as np
import pandas as pd
import sklearn

# libraries for text cleaning
import contractions
from bs4 import BeautifulSoup
from textblob import TextBlob
from spellchecker import SpellChecker
#from profanity_check import predict

#for oversampling
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

# libraries and packages for text (pre-)processing 
import string
import re
import nltk

from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import *
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk import pos_tag_sents
from nltk.tokenize import sent_tokenize

##libraries and packages for EDA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.01 µs


In [4]:
train_df = pd.read_csv("Data/train.csv")
print(train_df.shape)
train_df.head()

(159571, 8)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


<a id="textcleaning"></a>
# 1. Text Cleaning

## Convert to Lower Case

We convert all letters to lower case to prepare for the following steps of text cleaning. Exceptional cases such as capital abbreviation will be solved by replacing typos, slang, acronyms or informal abbreviations technique in the subsquent steps.

In [3]:
train_df["clean_text"] = train_df["comment_text"].apply(lambda x: x.lower())
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_text
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation\nwhy the edits made under my usern...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,d'aww! he matches this background colour i'm s...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"hey man, i'm really not trying to edit war. it..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"""\nmore\ni can't make any real suggestions on ..."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"you, sir, are my hero. any chance you remember..."


## Expand Contractions


Contractions are words or combinations of words that are shortened by dropping letters and replacing them by an apostrophe. Removing contractions helps contribute to text standardization. We use contractions package to expand contractions.

In [4]:
train_df["clean_text"] = train_df["clean_text"].apply(lambda x: contractions.fix(x))

In [5]:
# check if expand contractions works
print("Original text: \n", train_df["comment_text"][2])
print("Clean text: \n", train_df["clean_text"][2])

Original text: 
 Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.
Clean text: 
 hey man, i am really not trying to edit war. it is just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. he seems to care more about the formatting than the actual info.


## Remove Noise

Remove unnecessary characters or punctuation such as URLs, HTML tags, non-ASCII characters, or other special characters 

### Remove URL

In [6]:
# replace URL with space
train_df["clean_text"] = train_df["clean_text"].apply(lambda x: re.sub(r'https?://\S+|www\.\S+', ' ', x))

###  Remove Non-ASCI Characters

In [7]:
# replace Non_ASCI characters with space
train_df["clean_text"] = train_df["clean_text"].apply(lambda x: re.sub(r'[^\x00-\x7f]', ' ', x))

###  Remove Special Characters

In [8]:
regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        u"\ufe0f"  # dingbats
        "]+", flags = re.UNICODE)

In [9]:
# replace special characters with space
train_df["clean_text"] = train_df["clean_text"].apply(lambda x: regrex_pattern.sub(' ', x))

In [10]:
# check if special characters are removed
print("Original text: \n", train_df["comment_text"][143])
print("Clean text: \n", train_df["clean_text"][143])

Original text: 
 "P.S. It's not polite to talk to people behind their backs, please remove your comments from Mrph's talk page.

Vaughan
You're right; I went to check your previous edit and found a page on the Marvel site that spelled it ""Vaughn"", but now I am finding many more that spell it correctly. Thanks for the edits.   (☎☓) 

"
Clean text: 
 "p.s. it is not polite to talk to people behind their backs, please remove your comments from mrph's talk page.

vaughan
you are right; i went to check your previous edit and found a page on the marvel site that spelled it ""vaughn"", but now i am finding many more that spell it correctly. thanks for the edits.   (  ) 

"


### Remove HTML Tag (BeautifulSoup not really useful? merely remove space?)

In [11]:
cleaned_text = train_df["clean_text"].apply(lambda x: BeautifulSoup(str(x)).get_text())

In [12]:
text_changed = cleaned_text!=train_df["clean_text"]

In [13]:
[i for i, x in enumerate(text_changed) if x][:10]

[569, 1403, 2028, 2189, 3458, 3500, 3687, 4001, 4030, 4090]

In [14]:
train_df["clean_text"][228]

'   heritage from village           in macedonian          . sources claim that the village was pure slavic.'

In [15]:
cleaned_text[228]

'   heritage from village           in macedonian          . sources claim that the village was pure slavic.'

In [16]:
# replace HTML tag with space
html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
train_df["clean_text"] = train_df["clean_text"].apply(lambda x: re.sub(html, " ", x))

###  Remove Extra Space

In [17]:
# replace \n with space
train_df["clean_text"] = train_df["clean_text"].apply(lambda x: re.sub('\n', ' ', x))

In [18]:
# replace \r\n with space
train_df["clean_text"] = train_df["clean_text"].apply(lambda x: re.sub('\r\n', ' ', x))

In [19]:
# remove extra space
train_df["clean_text"] = train_df["clean_text"].apply(lambda x: re.sub(' +', ' ', x))

## Replace Common Slangs

Slang, acronyms or informal abbreviations should be replaced with formal English. The list of common slangs used in Tweets takes reference from https://www.kaggle.com/code/nmaguette/up-to-date-list-of-slangs-for-text-preprocessing.

In [20]:
# read abbreviation.csv
abbreviations = pd.read_csv('Data/abbreviations.csv')
abbreviations.head()

,abbreviation,translation
0,$,dollar
1,€,euro
2,4ao,for adults only
3,a.m,before midday
4,a3,anytime anywhere anyplace


In [21]:
# convert the data frame to a dictionary
abbreviations_dict = dict(zip(abbreviations.abbreviation, abbreviations.translation))

In [22]:
# define a helper function to replace the abbreviations
def convert_abbrev(text):
    # create a pattern of all abbreviations and make sure they are not part of a longer word
    abbreviations_pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in abbreviations_dict.keys()) + r')(?!\w)')
    # replace an abbreviation with its translation
    text = abbreviations_pattern.sub(lambda x: abbreviations_dict[x.group()], text)
    return text

In [23]:
# replace the slangs
train_df["clean_text"] = train_df["clean_text"].apply(convert_abbrev)

In [24]:
# check if slangs are replaced
print("Original text: \n", train_df["comment_text"][1])
print("Clean text: \n", train_df["clean_text"][1])

Original text: 
 D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)
Clean text: 
 d'aww! he matches this background colour i am seemingly stuck with. thanks. (talk) 21:51, january 11, 2016 (coordinated universal time)


## Spelling Correction

We should correct the misspellings in the text. Both SpellChecker and TextBlob provide such functions, and we would like to compare their performance.

In [25]:
# select random texts from clean_text
length = len(train_df["clean_text"])
random_num = random.sample(range(length), 100)
random_text = train_df["clean_text"][random_num]

In [26]:
# using TextBlob package
start_time1 = time.time()
random_text.apply(lambda x: TextBlob(x).correct())
print("--- %s seconds ---" % (time.time() - start_time1))

--- 30.84517478942871 seconds ---


In [27]:
# using SpellChecker package
start_time2 = time.time()
random_text.apply(lambda x: SpellChecker().correction(x))
print("--- %s seconds ---" % (time.time() - start_time2))

--- 16.331270933151245 seconds ---


Randomly select 100 texts and apply spelling correction functions on them. Comparing the execution time of 2 different packages, SpellChecker is much faster than TextBlob. Considering we are using a large-scale dataset, SpellChecker is preferred.

In [28]:
def correct_spelling(text):
    start_time = time.time()
    cleaned_text = []
    spellchecker = SpellChecker()
    for i in range(text.shape[0]):
        if i%100==0:
            print(f'{i}-th text is being processed')
        cleaned_text.append(spellchecker.correction(text[i]))
    print("--- %s seconds ---" % (time.time() - start_time))
    return cleaned_text

In [29]:
cleaned_text = correct_spelling(train_df["clean_text"][:1001])

0-th text is being processed
100-th text is being processed
200-th text is being processed
300-th text is being processed
400-th text is being processed
500-th text is being processed
600-th text is being processed
700-th text is being processed
800-th text is being processed
900-th text is being processed
1000-th text is being processed
--- 67.02877187728882 seconds ---


In [30]:
train_df["clean_text"][:1001].index[cleaned_text!=train_df["clean_text"][:1001]]

Int64Index([ 62,  89, 101, 173, 175, 211, 217, 223, 226, 241, 250, 254, 259,
            268, 276, 299, 320, 323, 376, 381, 397, 408, 423, 448, 465, 470,
            504, 545, 592, 627, 632, 646, 715, 743, 758, 787, 806, 807, 814,
            823, 831, 844, 852, 874, 877, 883, 897, 899, 913, 923, 947, 971],
           dtype='int64')

In [31]:
train_df["clean_text"][971]

'. fu ck ing trollreasons'

In [32]:
cleaned_text[971]

However, many corrections do not make sense, and may omit some useful information. We decided not to use established package to perform spelling correction.

## Remove Punctuations

We remove punctuations from the text as the final step of text cleaning.

In [33]:
# # remove punctuations
#train_df["clean_text"] = train_df["clean_text"].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
r = re.compile(r'''([!#$%&'()*+,./:;<=>?@[\]^_`{|}~-])[!"#$%&'()*+,./:;<=>?@[\]^_`{|}~-]+''')
train_df["clean_text"] = train_df["clean_text"].apply(lambda x: r.sub(r'\1', x))

In [34]:
train_df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_text
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation why the edits made under my userna...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,d'aww! he matches this background colour i am ...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"hey man, i am really not trying to edit war. i..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,""" more i cannot make any real suggestions on i..."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"you, sir, are my hero. any chance you remember..."
...,...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0,""":and for the second time of asking, when your..."
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0,you should be ashamed of yourself that is a ho...
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0,"spitzer umm, there is no actual article for pr..."
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0,and it looks like it was actually you who put ...


In [35]:
train_df.drop('comment_text', axis=1).to_csv('Data/cleaned_train.csv', index=False)

<a id="textpreprocessing"></a>
# 2. Text Preprocessing

In [36]:
#cleaned_df = pd.read_csv("/cleaned_train.csv")
cleaned_df = pd.read_csv("Data/cleaned_train.csv")
cleaned_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_text
0,0000997932d777bf,0,0,0,0,0,0,explanation why the edits made under my userna...
1,000103f0d9cfb60f,0,0,0,0,0,0,d'aww! he matches this background colour i am ...
2,000113f07ec002fd,0,0,0,0,0,0,"hey man, i am really not trying to edit war. i..."
3,0001b41b1c6bb37e,0,0,0,0,0,0,""" more i cannot make any real suggestions on i..."
4,0001d958c54c6e35,0,0,0,0,0,0,"you, sir, are my hero. any chance you remember..."


## Tokenization (NLTK has a package specially catered to tokenizing tweets)

In [37]:
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
cleaned_df['text_tokenized'] = cleaned_df['clean_text'].apply(tt.tokenize)

In [38]:
cleaned_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_text,text_tokenized
0,0000997932d777bf,0,0,0,0,0,0,explanation why the edits made under my userna...,"[explanation, why, the, edits, made, under, my..."
1,000103f0d9cfb60f,0,0,0,0,0,0,d'aww! he matches this background colour i am ...,"[d'aww, !, he, matches, this, background, colo..."
2,000113f07ec002fd,0,0,0,0,0,0,"hey man, i am really not trying to edit war. i...","[hey, man, ,, i, am, really, not, trying, to, ..."
3,0001b41b1c6bb37e,0,0,0,0,0,0,""" more i cannot make any real suggestions on i...","["", more, i, cannot, make, any, real, suggesti..."
4,0001d958c54c6e35,0,0,0,0,0,0,"you, sir, are my hero. any chance you remember...","[you, ,, sir, ,, are, my, hero, ., any, chance..."


## Remove Stop Words (or/and Frequent words/ Rare words)

### Stop Words Removal

In [39]:
from nltk.corpus import stopwords

#review all stop words in the library
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [40]:
#remove stop words from tokenized text
stop_words = stopwords.words('english')

def remove_stopwords(row):
    # check in lowercase 
    t = [token for token in row['text_tokenized'] if token.lower() not in stop_words]
    text = ' '.join(t)    
    return pd.Series([text])    #for stemming

cleaned_df[['text_no_stop_words']] = cleaned_df.apply(remove_stopwords, axis = 1)


cleaned_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_text,text_tokenized,text_no_stop_words
0,0000997932d777bf,0,0,0,0,0,0,explanation why the edits made under my userna...,"[explanation, why, the, edits, made, under, my...",explanation edits made username hardcore metal...
1,000103f0d9cfb60f,0,0,0,0,0,0,d'aww! he matches this background colour i am ...,"[d'aww, !, he, matches, this, background, colo...",d'aww ! matches background colour seemingly st...
2,000113f07ec002fd,0,0,0,0,0,0,"hey man, i am really not trying to edit war. i...","[hey, man, ,, i, am, really, not, trying, to, ...","hey man , really trying edit war . guy constan..."
3,0001b41b1c6bb37e,0,0,0,0,0,0,""" more i cannot make any real suggestions on i...","["", more, i, cannot, make, any, real, suggesti...",""" cannot make real suggestions improvement - w..."
4,0001d958c54c6e35,0,0,0,0,0,0,"you, sir, are my hero. any chance you remember...","[you, ,, sir, ,, are, my, hero, ., any, chance...",", sir , hero . chance remember page ?"


In [41]:
#drop clean_text and tokenized column
#del cleaned_df['clean_text']
#del cleaned_df['text_tokenized']
cleaned_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_text,text_tokenized,text_no_stop_words
0,0000997932d777bf,0,0,0,0,0,0,explanation why the edits made under my userna...,"[explanation, why, the, edits, made, under, my...",explanation edits made username hardcore metal...
1,000103f0d9cfb60f,0,0,0,0,0,0,d'aww! he matches this background colour i am ...,"[d'aww, !, he, matches, this, background, colo...",d'aww ! matches background colour seemingly st...
2,000113f07ec002fd,0,0,0,0,0,0,"hey man, i am really not trying to edit war. i...","[hey, man, ,, i, am, really, not, trying, to, ...","hey man , really trying edit war . guy constan..."
3,0001b41b1c6bb37e,0,0,0,0,0,0,""" more i cannot make any real suggestions on i...","["", more, i, cannot, make, any, real, suggesti...",""" cannot make real suggestions improvement - w..."
4,0001d958c54c6e35,0,0,0,0,0,0,"you, sir, are my hero. any chance you remember...","[you, ,, sir, ,, are, my, hero, ., any, chance...",", sir , hero . chance remember page ?"


## Stemming

We will use Snowball Stemmer to realise stemming. Comparing with Poster Stemmer, it is more efficient and has higher performance. Comparing with Lancaster Stemmer, it is less aggressive and can keep more word meanings for the Semantic Analysis in our later stage. 

In [42]:
stemmer = SnowballStemmer(language = 'english')
def stem_list_of_words(words):
    stemmed_words = []
    for word in words:
        stemmed_words.append(stemmer.stem(word))
    stemmed_sent = ' '.join(stemmed_words)  
    return stemmed_sent
        

cleaned_df['stemmed'] = cleaned_df['text_tokenized'].apply(stem_list_of_words)
#del cleaned_df['for_stemming_use']
cleaned_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_text,text_tokenized,text_no_stop_words,stemmed
0,0000997932d777bf,0,0,0,0,0,0,explanation why the edits made under my userna...,"[explanation, why, the, edits, made, under, my...",explanation edits made username hardcore metal...,explan whi the edit made under my usernam hard...
1,000103f0d9cfb60f,0,0,0,0,0,0,d'aww! he matches this background colour i am ...,"[d'aww, !, he, matches, this, background, colo...",d'aww ! matches background colour seemingly st...,d'aww ! he match this background colour i am s...
2,000113f07ec002fd,0,0,0,0,0,0,"hey man, i am really not trying to edit war. i...","[hey, man, ,, i, am, really, not, trying, to, ...","hey man , really trying edit war . guy constan...","hey man , i am realli not tri to edit war . it..."
3,0001b41b1c6bb37e,0,0,0,0,0,0,""" more i cannot make any real suggestions on i...","["", more, i, cannot, make, any, real, suggesti...",""" cannot make real suggestions improvement - w...",""" more i cannot make ani real suggest on impro..."
4,0001d958c54c6e35,0,0,0,0,0,0,"you, sir, are my hero. any chance you remember...","[you, ,, sir, ,, are, my, hero, ., any, chance...",", sir , hero . chance remember page ?","you , sir , are my hero . ani chanc you rememb..."


## Part of Speech Tagging (POS Tagging)

In [43]:
def to_word_tokens(sent_tokens):
    word_tokens = [] 
    for sent_token in sent_tokens:
        word_tokens.append(tt.tokenize(sent_token))
    return word_tokens

In [45]:
cleaned_df['for_tagging_use_sent_token'] = cleaned_df['clean_text'].apply(sent_tokenize)
cleaned_df['for_tagging_use_word_token'] = cleaned_df['for_tagging_use_sent_token'].apply(to_word_tokens)
cleaned_df['POS_tagging'] = cleaned_df['for_tagging_use_word_token'].apply(pos_tag_sents)
cleaned_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_text,text_tokenized,text_no_stop_words,stemmed,for_tagging_use_sent_token,for_tagging_use_word_token,POS_tagging
0,0000997932d777bf,0,0,0,0,0,0,explanation why the edits made under my userna...,"[explanation, why, the, edits, made, under, my...",explanation edits made username hardcore metal...,explan whi the edit made under my usernam hard...,[explanation why the edits made under my usern...,"[[explanation, why, the, edits, made, under, m...","[[(explanation, NN), (why, WRB), (the, DT), (e..."
1,000103f0d9cfb60f,0,0,0,0,0,0,d'aww! he matches this background colour i am ...,"[d'aww, !, he, matches, this, background, colo...",d'aww ! matches background colour seemingly st...,d'aww ! he match this background colour i am s...,"[d'aww!, he matches this background colour i a...","[[d'aww, !], [he, matches, this, background, c...","[[(d'aww, NN), (!, .)], [(he, PRP), (matches, ..."
2,000113f07ec002fd,0,0,0,0,0,0,"hey man, i am really not trying to edit war. i...","[hey, man, ,, i, am, really, not, trying, to, ...","hey man , really trying edit war . guy constan...","hey man , i am realli not tri to edit war . it...","[hey man, i am really not trying to edit war.,...","[[hey, man, ,, i, am, really, not, trying, to,...","[[(hey, NN), (man, NN), (,, ,), (i, JJ), (am, ..."
3,0001b41b1c6bb37e,0,0,0,0,0,0,""" more i cannot make any real suggestions on i...","["", more, i, cannot, make, any, real, suggesti...",""" cannot make real suggestions improvement - w...",""" more i cannot make ani real suggest on impro...","["" more i cannot make any real suggestions on ...","[["", more, i, cannot, make, any, real, suggest...","[[("", IN), (more, JJR), (i, JJ), (cannot, NNS)..."
4,0001d958c54c6e35,0,0,0,0,0,0,"you, sir, are my hero. any chance you remember...","[you, ,, sir, ,, are, my, hero, ., any, chance...",", sir , hero . chance remember page ?","you , sir , are my hero . ani chanc you rememb...","[you, sir, are my hero., any chance you rememb...","[[you, ,, sir, ,, are, my, hero, .], [any, cha...","[[(you, PRP), (,, ,), (sir, VB), (,, ,), (are,..."


In [ ]:
cleaned_df = cleaned_df.drop(columns=['for_tagging_use_sent_token','for_tagging_use_word_token'])
cleaned_df.head()

## Lemmatization

In [ ]:
cleaned_df['POS_tagging_flat'] = cleaned_df['POS_tagging'].apply(lambda x: [element for innerList in x for element in innerList])

In [ ]:
lemmatizer = WordNetLemmatizer()
cleaned_df['lemmatization'] = cleaned_df['POS_tagging_flat'].apply(lambda x: [lemmatizer.lemmatize(word) for word, tag in x 
                                                                              if ((tag.startswith('JJ') or tag.startswith('NN') or tag.startswith('RB') or tag.startswith('VB')) and (word not in string.punctuation))])

Stemming might not be useful for creating features. Lemmatization based on tagging result has achieved same result as text without stop words. Text_tokenized column, POS tagging result, and lemmatization for future analysis.

In [ ]:
del cleaned_df['clean_text']
del cleaned_df['text_no_stop_words']
del cleaned_df['stemmed']
del cleaned_df['POS_tagging']
cleaned_df.head()

In [ ]:
cleaned_df.to_csv('Data/processed_train.csv', index=False)

<a id="EDA"></a>
# 3. EDA

In [5]:
# prevent lists from being converted to strings
processed_df = pd.read_csv('Data/processed_train.csv', 
                           converters={'text_tokenized': literal_eval, 'POS_tagging_flat': literal_eval, 'lemmatization': literal_eval})
processed_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,text_tokenized,POS_tagging_flat,lemmatization
0,0000997932d777bf,0,0,0,0,0,0,"[explanation, why, the, edits, made, under, my...","[(explanation, NN), (why, WRB), (the, DT), (ed...","[explanation, edits, made, username, hardcore,..."
1,000103f0d9cfb60f,0,0,0,0,0,0,"[d'aww, !, he, matches, this, background, colo...","[(d'aww, NN), (!, .), (he, PRP), (matches, VBZ...","[d'aww, match, colour, i, am, seemingly, stuck..."
2,000113f07ec002fd,0,0,0,0,0,0,"[hey, man, ,, i, am, really, not, trying, to, ...","[(hey, NN), (man, NN), (,, ,), (i, JJ), (am, V...","[hey, man, i, am, really, not, trying, edit, w..."
3,0001b41b1c6bb37e,0,0,0,0,0,0,"["", more, i, cannot, make, any, real, suggesti...","[("", IN), (more, JJR), (i, JJ), (cannot, NNS),...","[more, i, cannot, make, real, suggestion, impr..."
4,0001d958c54c6e35,0,0,0,0,0,0,"[you, ,, sir, ,, are, my, hero, ., any, chance...","[(you, PRP), (,, ,), (sir, VB), (,, ,), (are, ...","[sir, are, hero, chance, remember, page, is]"


In [ ]:
colors_ = ["deep blue","blue", "teal", "lightblue","darkblue","purple"]

palette= sns.xkcd_palette(colors_)

x = processed_df.iloc[:,1:7].sum()

plt.figure(figsize=(15,8))

ax= sns.barplot(x=x.index,y=x.values,palette=palette)
plt.title("Comment Category Distribution in Training set")
plt.xlabel('Label ')
plt.ylabel('No. of tweets', fontsize=11)

patch = ax.patches
labels = x.values
for rect, label in zip(patch, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 10, label, va='bottom',ha='center')

plt.show()

In [ ]:
# Obtain counts of each class
counts = {
    '0': [],
    '1': [],
}

categories = list(processed_df.columns.values)
categories = categories[1:7]

for c in categories:
    count_occurence_1 = len(processed_df.loc[processed_df[c] == 1])
    count_occurence_0 = len(processed_df.loc[processed_df[c] == 0])
    counts['0'].append(count_occurence_0)
    counts['1'].append(count_occurence_1)

# Plot distribution of comment category
x = np.arange(len(categories))  # the label locations
width = 0.25  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(figsize=(15,8))

for attribute, measurement in counts.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Number of Tweets')
ax.set_xlabel('Comment Category')
ax.set_title('Distribution of Comment Category')
ax.set_xticks(x + width)
ax.set_xticklabels(categories)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

In [ ]:
toxic_comments_count= len(processed_df[processed_df[categories].sum(axis=1) > 0])
nontoxic_comments_count = len(processed_df[processed_df[categories].sum(axis=1) == 0])

print("Total number of comments = ",len(processed_df))
print("Number of non-toxic comments = ",nontoxic_comments_count)
print("Number of comments with toxic labels =",(len(processed_df)-nontoxic_comments_count))

print(f"{round(nontoxic_comments_count/len(processed_df) * 100,3)} % percentage of rows contains only zeros in training data")

count = pd.DataFrame(dict(
    types = ['toxic', 'non-toxic'],
    counts = [toxic_comments_count, nontoxic_comments_count]
))

sns.barplot(data=count, x='types',y='counts').set(title='Toxic vs Non-Toxic Tweets Distribution')

From the plot above, we also observe that the dataset is highly imbalanced. This fact is also supported by the the big number of difference in toxic and non-toxic comment (about 90% of training data is non-toxic). Therefore, in the later part of the project, we will perform oversampling on the training set. 

In [ ]:
# Oversampling
ros = RandomOverSampler(random_state=0)

toxic_comments = processed_df[processed_df[categories].sum(axis=1) > 0]
nontoxic_comments = processed_df[processed_df[categories].sum(axis=1) == 0]

toxic_comments['class'] = 1 #1 for toxic
nontoxic_comments['class'] = 0

processed_df_new = pd.concat([toxic_comments, nontoxic_comments], ignore_index=True, axis=0)

X_train = processed_df_new.drop('class', axis = 1)
y_train = processed_df_new['class']
X_train_ros, y_train_ros= ros.fit_resample(X_train, y_train)
                              
# Check the number of records after over sampling
print(X_train_ros.shape)
print(sorted(Counter(y_train_ros).items()))
                              
#Concatenate X_train and y_train
train_oversampled = pd.concat([X_train_ros,  y_train_ros], axis=1)

In [ ]:
train_toxic_comments = train_oversampled[train_oversampled['class']== 1]
train_clean_comments = train_oversampled[train_oversampled['class']== 0]

count = pd.DataFrame(dict(
    types = ['toxic', 'non-toxic'],
    counts = [len(train_toxic_comments), len(train_clean_comments)]
))

sns.barplot(data=count, x='types',y='counts').set(title='Toxic vs Non-Toxic Tweets Distribution')

In [ ]:
train_oversampled.to_csv('Data/train_oversampled.csv', index=False)

## Word Length Distribution

In [ ]:
train_oversampled['words length'] = train_oversampled['text_tokenized'].apply(len)

In [ ]:
px.histogram(train_oversampled, x='words length', title='Word Length Distribution')

## Characters Length Distribution

In [ ]:
train_oversampled['characters length'] = train_oversampled['text_tokenized'].apply(lambda x: ''.join(word for word in x)).apply(lambda x: len(x))

In [ ]:
px.histogram(train_oversampled, x='words length', title='Characters Length Distribution')

## Top frequent tags (using POS result) 

In [51]:
processed_df= pd.read_csv("Data/train_oversampled.csv",
                           converters={'text_tokenized': literal_eval, 'POS_tagging_flat': literal_eval, 'lemmatization': literal_eval})
processed_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,text_tokenized,POS_tagging_flat,lemmatization,class
0,0002bcb3da6cb337,1,1,1,0,1,0,"[cocksucker, before, you, piss, around, on, my...","[(cocksucker, NN), (before, IN), (you, PRP), (...","[cocksucker, piss, around, work]",1
1,0005c987bdfc9d4b,1,0,0,0,0,0,"[hey, ., what, is, it, ., @, |, talk, ., what,...","[(hey, NN), (., .), (what, WP), (is, VBZ), (it...","[hey, is, talk, is, exclusive, group, wp, tali...",1
2,0007e25b2121310b,1,0,0,0,0,0,"[bye, !, do, not, look, ,, come, or, think, of...","[(bye, NN), (!, .), (do, VB), (not, RB), (look...","[bye, do, not, look, come, think, comming, bac...",1
3,001810bf8c45bf5f,1,0,1,0,1,1,"[you, are, gay, or, antisemmitian, ?, archange...","[(you, PRP), (are, VBP), (gay, JJ), (or, CC), ...","[are, gay, antisemmitian, archangel, white, ti...",1
4,00190820581d90ce,1,0,1,0,1,0,"[fuck, your, filthy, mother, in, the, ass, ,, ...","[(fuck, VB), (your, PRP$), (filthy, JJ), (moth...","[fuck, filthy, mother, as, dry]",1


### Overall data

In [ ]:
POS_list = []

for x in processed_df['POS_tagging_flat']:
    POS_list.append(x)

In [ ]:
from collections import Counter
from itertools import chain

POS_count = Counter(chain(*POS_list))

In [ ]:
POS_frequent_df = pd.DataFrame.from_dict(POS_count, orient='index').reset_index()
POS_frequent_df.columns = ['POS_tags', 'Count']
POS_frequent_df = POS_frequent_df.sort_values(by=['Count'], ascending=False)
POS_frequent_df['Word'] = POS_frequent_df['POS_tags'].apply(lambda x: x[0])
POS_frequent_df['Tag'] = POS_frequent_df['POS_tags'].apply(lambda x: x[1])
POS_frequent_df

In [ ]:
list_punctuations = []
punctuations = string.punctuation 
for x in punctuations:
    list_punctuations.append(x)
    
POS_frequent_df = POS_frequent_df.query("Word not in @stop_words & Word not in @list_punctuations")
POS_frequent_df

In [ ]:
fig, ax = plt.subplots()
fig.suptitle("Top frequent tags", fontsize=12)
sns.barplot(x='Count', y='Word', hue='Tag', 
            data=POS_frequent_df.iloc[:20,:], dodge=False, ax=ax)
ax.grid(axis="x")
plt.show()

### Toxic Comments Data

In [ ]:
toxic_POS_list = []

for x in processed_df['POS_tagging_flat'][((processed_df.toxic == 1) | (processed_df.severe_toxic == 1) | (processed_df.obscene == 1) | (processed_df.threat == 1) | (processed_df.identity_hate == 1))]:
    toxic_POS_list.append(x)

In [ ]:
toxic_POS_count = Counter(chain(*toxic_POS_list))

In [ ]:
toxic_POS_frequent_df = pd.DataFrame.from_dict(toxic_POS_count, orient='index').reset_index()
toxic_POS_frequent_df.columns = ['toxic_POS_tags', 'Count']
toxic_POS_frequent_df = toxic_POS_frequent_df.sort_values(by=['Count'], ascending=False)
toxic_POS_frequent_df['Word'] = toxic_POS_frequent_df['toxic_POS_tags'].apply(lambda x: x[0])
toxic_POS_frequent_df['Tag'] = toxic_POS_frequent_df['toxic_POS_tags'].apply(lambda x: x[1])

In [ ]:
toxic_POS_frequent_df = toxic_POS_frequent_df.query("Word not in @stop_words & Word not in @list_punctuations")
toxic_POS_frequent_df

In [ ]:
fig, ax = plt.subplots()
fig.suptitle("Top frequent toxic tags", fontsize=12)
sns.barplot(x='Count', y='Word', hue='Tag', 
            data=toxic_POS_frequent_df.iloc[:20,:], dodge=False, ax=ax)
ax.grid(axis="x")
plt.show()

### Non-Toxic Comments Data

In [ ]:
nontoxic_POS_list = []

for x in processed_df['POS_tagging_flat'][((processed_df.toxic == 0) &(processed_df.severe_toxic == 0) & (processed_df.obscene == 0) & (processed_df.threat == 0) & (processed_df.identity_hate == 0))]:
    nontoxic_POS_list.append(x)

In [ ]:
nontoxic_POS_count = Counter(chain(*nontoxic_POS_list))

In [ ]:
nontoxic_POS_frequent_df = pd.DataFrame.from_dict(nontoxic_POS_count, orient='index').reset_index()
nontoxic_POS_frequent_df.columns = ['nontoxic_POS_tags', 'Count']
nontoxic_POS_frequent_df = nontoxic_POS_frequent_df.sort_values(by=['Count'], ascending=False)
nontoxic_POS_frequent_df['Word'] = nontoxic_POS_frequent_df['nontoxic_POS_tags'].apply(lambda x: x[0])
nontoxic_POS_frequent_df['Tag'] = nontoxic_POS_frequent_df['nontoxic_POS_tags'].apply(lambda x: x[1])

In [ ]:
nontoxic_POS_frequent_df = nontoxic_POS_frequent_df.query("Word not in @stop_words & Word not in @list_punctuations")
nontoxic_POS_frequent_df

In [ ]:
fig, ax = plt.subplots()
fig.suptitle("Top frequent non-toxic tags", fontsize=12)
sns.barplot(x='Count', y='Word', hue='Tag', 
            data=nontoxic_POS_frequent_df.iloc[:20,:], dodge=False, ax=ax)
ax.grid(axis="x")
plt.show()

## Weighted Words - Bag of Words (BoW) - Bag of n-grams

### Frequent Words Identification

In [ ]:
from nltk.probability import FreqDist
def most_frequent_words(text):
    words = word_tokenize(text)
    fdist = FreqDist(words) 
    
    df_fdist = pd.DataFrame({'Word': fdist.keys(),
                             'Frequency': fdist.values()})
    df_fdist = df_fdist.sort_values(by='Frequency', ascending=False)
    
    return df_fdist.head(50)

Remove stopwords & punctuaion

In [ ]:
stop_words = stopwords.words('english')
processed_df['lemmatization'] = processed_df['lemmatization'].apply(lambda x: [ i for i in x if (i not in string.punctuation) & (i not in stop_words)])

In [ ]:
processed_df

In [ ]:
#identify frequent words

combined_toxic = processed_df['lemmatization'][((processed_df.toxic == 1) | (processed_df.severe_toxic == 1) | (processed_df.obscene == 1) | (processed_df.threat == 1) | (processed_df.identity_hate == 1))].str.join(' ')
combined_toxic = ' '.join(combined_toxic)
toxic_frequent = most_frequent_words(combined_toxic)

In [ ]:
#identify frequent words
combined_nontoxic = processed_df['lemmatization'][((processed_df.toxic == 0) &(processed_df.severe_toxic == 0) & (processed_df.obscene == 0) & (processed_df.threat == 0) & (processed_df.identity_hate == 0))].str.join(' ')
combined_nontoxic = ' '.join(combined_nontoxic)
nontoxic_frequent = most_frequent_words(combined_nontoxic)

In [ ]:
toxic_frequent.head()

In [ ]:
nontoxic_frequent.head()

For toxic comments

In [ ]:
plt.figure(figsize=(12,8))
word_cloud = WordCloud(
                          background_color='black',
                          max_font_size = 80
                         ).generate(" ".join(toxic_frequent['Word']))
plt.imshow(word_cloud)
plt.axis('off')
plt.title("Wordcloud for toxic comments")
plt.show()

For non toxic comment

In [ ]:
plt.figure(figsize=(12,8))
word_cloud = WordCloud(
                          background_color='white',
                          max_font_size = 80,
                          colormap='rainbow'
                         ).generate(" ".join(nontoxic_frequent['Word']))
plt.imshow(word_cloud)
plt.title("Wordcloud for non-toxic comments")
plt.axis('off')
plt.show()

### Frequency Vector

Bigrams frequency

In [ ]:
vec = CountVectorizer(ngram_range=(2, 2))
bow = vec.fit_transform(processed_df['text_tokenized'].str.join(' '))
sum_of_words = bow.sum(axis=0)
bigrams_freq = [(word, sum_of_words[0, idx]) for word, idx in vec.vocabulary_.items()]
bigrams_freq =sorted(bigrams_freq, key = lambda x: x[1], reverse=True)
bigrams_freq = pd.DataFrame(bigrams_freq[:50], columns=["bigrams", "frequency"])

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(data = bigrams_freq, x = "frequency", y="bigrams")
plt.title("Bigrams frequency")

Trigrams frequency

In [ ]:
vec = CountVectorizer(ngram_range=(3, 3))
bow = vec.fit_transform(processed_df['text_tokenized'].str.join(' '))
sum_of_words = bow.sum(axis=0)
trigrams_freq = [(word, sum_of_words[0, idx]) for word, idx in vec.vocabulary_.items()]
trigrams_freq =sorted(trigrams_freq, key = lambda x: x[1], reverse=True)
trigrams_freq = pd.DataFrame(trigrams_freq[:50], columns=["trigrams", "frequency"])

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(data = trigrams_freq, x = "frequency", y="trigrams")
plt.title("Trigrams frequency")

### Term Frequency-Inverse Document Frequency (TF-IDF)

In [ ]:
tfidf =  TfidfVectorizer(ngram_range=(2, 2))
bow = tfidf.fit_transform(processed_df['text_tokenized'].str.join(' '))
sum_of_words = bow.sum(axis=0)
bigrams_freq = [(word, sum_of_words[0, idx]) for word, idx in tfidf.vocabulary_.items()]
bigrams_freq =sorted(bigrams_freq, key = lambda x: x[1], reverse=True)
bigrams_freq = pd.DataFrame(bigrams_freq[:50], columns=["bigrams", "frequency"])

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(data = bigrams_freq, x = "frequency", y="bigrams")
plt.title("Bigrams frequency in TF-IDF")

In [ ]:
tfidf =  TfidfVectorizer(ngram_range=(3, 3))
bow = tfidf.fit_transform(processed_df['text_tokenized'].str.join(' '))
sum_of_words = bow.sum(axis=0)
trigrams_freq = [(word, sum_of_words[0, idx]) for word, idx in tfidf.vocabulary_.items()]
trigrams_freq =sorted(trigrams_freq, key = lambda x: x[1], reverse=True)
trigrams_freq = pd.DataFrame(trigrams_freq[:50], columns=["trigrams", "frequency"])

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(data = trigrams_freq, x = "frequency", y="trigrams")
plt.title("Trigrams frequency in TF-IDF")

## Sentiment Analysis

In [ ]:
train_oversampled= pd.read_csv("Data/train_oversampled.csv")
train_oversampled.head()

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#calculate VADER score
sentiments = SentimentIntensityAnalyzer()
semtiment_df = processed_df[['id','toxic','severe_toxic','obscene','threat','insult','identity_hate','lemmatization']]
semtiment_df["Positive"] = [sentiments.polarity_scores(" ".join(i))["pos"] for i in semtiment_df["lemmatization"]]
semtiment_df["Negative"] = [sentiments.polarity_scores(" ".join(i))["neg"] for i in semtiment_df["lemmatization"]]
semtiment_df["Neutral"] = [sentiments.polarity_scores(" ".join(i))["neu"] for i in semtiment_df["lemmatization"]]
semtiment_df["compound"] = [sentiments.polarity_scores(" ".join(i))["compound"] for i in semtiment_df["lemmatization"]]
# semtiment_df = processed_df[["content", "Positive", "Negative", "Neutral"]]
semtiment_df

In [ ]:
senti_pie_count_list = []
senti_pie_count_list.append(len(semtiment_df[semtiment_df['compound'] < -0.05]))
senti_pie_count_list.append(len(semtiment_df[semtiment_df['compound'] > 0.05]))
senti_pie_count_list.append(len(semtiment_df[(semtiment_df['compound'] < 0.05) & (semtiment_df['compound'] > -0.05)]))

colors = ['pink', 'silver', 'steelblue']
labels = ["Negative","Positive","Neutral"]

plt.pie(np.array(senti_pie_count_list),labels = labels, colors = colors,autopct='%1.1f%%')
plt.title("Number of tweets in each sentiment category")
plt.legend()

In [ ]:
positive_words =' '.join([" ".join(i) for i in semtiment_df['lemmatization'][semtiment_df['compound'] > 0.05]])
# stopwords = set(STOPWORDS)
wordcloud = WordCloud(background_color="white",stopwords=set(STOPWORDS),collocations=False).generate(positive_words)
plt.figure( figsize=(15,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Frequent words in positive tweets")
plt.show()

In [ ]:
from wordcloud import WordCloud, STOPWORDS
negative_words =' '.join([" ".join(i) for i in semtiment_df['lemmatization'][semtiment_df['compound'] < -0.05]])
# stopwords = set(STOPWORDS)
wordcloud = WordCloud(background_color="white",colormap = 'inferno',stopwords=set(STOPWORDS),collocations=False).generate(negative_words)
plt.figure( figsize=(15,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Frequent words in negative tweets")
plt.show()

In [ ]:
# test relationship between negative VADER score with toxic comments
semtiment_df['negative']= np.where(semtiment_df['compound'] < -0.05, 1, 0)
contigency= pd.crosstab(semtiment_df['negative'], semtiment_df['toxic'])
contigency

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(contigency, annot=True, cmap="YlGnBu")

In [ ]:
from scipy.stats import chi2_contingency
# Chi-square test of independence
# H0 : the variables are independent, there is no relationship between the two categorical variables
# H1 : the variables are dependent, there is a relationship between the two categorical variables.
c, p, dof, expected = chi2_contingency(contigency)
if p < 0.05:
    print("p < alpha, reject H0")


<a id="featureengineering"></a>
# 4. Feature Engineering

## Word Embedding

There are some decisions we need to make:
- should stop words and punctuations be removed before embedding
- custom embeddings or pre-trained embeddings or embedding layer
- for pre-trained embeddings, which package to use: Word2Vec/GloVe, FastText, BERT
- the size of vector to represent each text
- word embedding or sentence embedding

In [ ]:
# for word embeddings
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec

# for BERT
import torch
from transformers import BertTokenizer, BertModel

In [ ]:
# define a helper function to calculate the embedding vector of each text
def get_embeddings(vectors, text_df, generate_missing=False, k=25):
    embeddings = []
    for i in range(len(text_df)):
        text = text_df[i]
        # dealing with empty text
        if len(text)<1:
            return np.zeros(k)
        # generate randomized vectors for unseen words if generate_missing is True
        if generate_missing:
            vectorized = [vectors[word][:k] if word in vectors else np.random.rand(k) for word in text]
        # represent unseen words with 0 vector if generate_missing is False
        else:
            vectorized = [vectors[word][:k] if word in vectors else np.zeros(k) for word in text]
        # each text is represented by averaging the vectors of its constituent words
        length = len(vectorized)
        summed = np.sum(vectorized, axis=0)
        averaged = np.divide(summed, length)
        embeddings.append(averaged)
    return embeddings

In [ ]:
# define a helper function to calculate the embedding vector of each text
def get_embeddings(vectors, text, generate_missing=False, k=25):
    # dealing with empty text
    if len(text)<1:
        return np.zeros(k)
    # generate randomized vectors for unseen words if generate_missing is True
    if generate_missing:
        vectorized = [vectors[word][:k] if word in vectors else np.random.rand(k) for word in text]
    # represent unseen words with 0 vector if generate_missing is False
    else:
        vectorized = [vectors[word][:k] if word in vectors else np.zeros(k) for word in text]
    # each text is represented by averaging the vectors of its constituent words
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

### Global Vectors for Word Representation (GloVe)

Similar to Word2Vec method, each word is presented by a high dimension vector and trained based on the surrounding words over a huge corpus. Since our project works on Tweets, we apply a pre-trained word embedding trained over Twitter content (https://github.com/stanfordnlp/GloVe).

Advantages:
- It captures the position of the words in the text (syntactic)
- It captures meaning in the words (semantics)

Limitations:
- It cannot capture the meaning of the word from the text (fails to capture polysemy)
- It cannot capture out-of-vocabulary words from corpus

In [ ]:
# generate a word2vec file used for model building
glove_input_file = "D:/GoogleDownloads/BT4222/glove.twitter.27B/glove.twitter.27B.25d.txt"
word2vec_output_file = "glove.twitter.27B.25d.txt.word2vec"
glove2word2vec(glove_input_file, word2vec_output_file)

In [ ]:
# build a GloVe model
glove_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False) 

In [ ]:
# generate embedding vectors of size 25 using tokenized text (stop words and punctuation kept)
# side note: map function outperforms for loop
embeddings_glove = processed_df["text_tokenized"].map(lambda x: get_embeddings(glove_model, x))

In [ ]:
# plot the result of embeddings
fig, axes = plt.subplots(nrows = 5, ncols=5, figsize=(15,15))
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
for dim in range(25):
    for label in labels:
        idx = processed_df[processed_df[label]==1].index.values
        li = list(map(lambda x: embeddings_glove[x][dim], idx))
        sns.kdeplot(li, fill=True, ax=axes[dim//5,dim%5])
    idx = processed_df[processed_df[labels].sum(axis=1)==0].index.values
    li = list(map(lambda x: embeddings_glove[x][dim], idx))
    sns.kdeplot(li, fill=True, ax=axes[dim//5,dim%5])

The difference among different labels is not very obvious.

In [ ]:
pd.DataFrame(embeddings_glove).to_csv('embeddings_glove.csv', index=False)

### FastText

Using FastText, each word, w, is represented as a bag of character n-gram. For example, given the word "introduce" and n = 3, FastText will produce the following representation composed of character tri-grams: < in, int, ntr, tro, rod, odu, duc, uce, ce > Note that the sequence, corresponding to the word here is different from the tri-gram "int" from the word introduce. Again, FastText is pre-trained on huge corpus (https://fasttext.cc/docs/en/english-vectors.html).

Advantages:
- Works for rare words, if their character n-grams which are still shared with other words
- Solves out of vocabulary words with n-gram in character level

Limitations:
- It cannot capture the meaning of the word from the text (fails to capture polysemy)

In [ ]:
# build a FastText model
start_time3 = time.time()
fasttext_path = "D:/GoogleDownloads/BT4222/crawl-300d-2M-subword/crawl-300d-2M-subword.vec"
fasttext_model = gensim.models.KeyedVectors.load_word2vec_format(fasttext_path, binary=False, limit=30000000000) # load 30B most common words
print("--- %s seconds ---" % (time.time() - start_time3))

In [ ]:
# generate embedding vectors of size 25 using tokenized text (stop words and punctuation kept)
embeddings_fasttext = processed_df["text_tokenized"].map(lambda x: get_embeddings(fasttext_model, x))

In [ ]:
# plot the result of embeddings
fig, axes = plt.subplots(nrows = 5, ncols=5, figsize=(15,15))
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
for dim in range(25):
    for label in labels:
        idx = processed_df[processed_df[label]==1].index.values
        li = list(map(lambda x: embeddings_fasttext[x][dim], idx))
        sns.kdeplot(li, fill=True, ax=axes[dim//5,dim%5])
    idx = processed_df[processed_df[labels].sum(axis=1)==0].index.values
    li = list(map(lambda x: embeddings_fasttext[x][dim], idx))
    sns.kdeplot(li, fill=True, ax=axes[dim//5,dim%5])

Difference in distribution among different labels is very slight.

In [ ]:
pd.DataFrame(embeddings_fasttext).to_csv('embeddings_fasttext.csv', index=False)

### Bidirectional Encoder Representations from Transformers (BERT)

While each word has a fixed representation under forementioned models regardless of the context within which the word appears, BERT produces word representations that are dynamically informed by the words around them. We use the BERT base model to create sentence embeddings.

Advantages:
-  It captures the meaning of the word from the text (incorporates context, handling polysemy)

Limitations:
- Computationally expensive
- It cannot capture out-of-vocabulary words from a corpus

In [ ]:
# define a helper function to calculate the embedding vector of each text
def get_BERT_embeddings(text, max_len=512): # BERT can only take up to 512 tokens
    # tokenize sentence with BERT tokenizer
    tokenized_text = tokenizer.tokenize(text)
    tokenized_text = tokenized_text[:max_len-2] # truncate overlength text
    # add special tokens
    tokenized_text = ["[CLS]"] + tokenized_text + ["[SEP]"]
    # map the token strings to their vocabulary indeces
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # create segment ID
    segments_ids = [1] * len(tokenized_text)
    # convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    # fead the inputs into BERT model
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]
    # use the average of the second last hidden layer of each token as sentence embedding
    token_vecs = hidden_states[-2][0]
    sentence_embedding = torch.mean(token_vecs, dim=0).numpy()
    return sentence_embedding

In [ ]:
# load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True)# whether the model returns all hidden-states
# put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

In [ ]:
# prepare the input used for BERT model
sentence_tokenized = processed_df['text_tokenized'].map(lambda x: ' '.join(x))

In [ ]:
# generate embedding vectors (stop words and punctuation kept)
start_time4 = time.time()
embeddings_bert = sentence_tokenized.map(get_BERT_embeddings)
print("--- %s seconds ---" % (time.time() - start_time4))

In [ ]:
# plot the result of embeddings
fig, axes = plt.subplots(nrows = 5, ncols=5, figsize=(15,15))
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
for dim in range(25):
    for label in labels:
        idx = processed_df[processed_df[label]==1].index.values
        li = list(map(lambda x: embeddings_bert[x][dim], idx))
        sns.kdeplot(li, fill=True, ax=axes[dim//5,dim%5])
    idx = processed_df[processed_df[labels].sum(axis=1)==0].index.values
    li = list(map(lambda x: embeddings_bert[x][dim], idx))
    sns.kdeplot(li, fill=True, ax=axes[dim//5,dim%5])

In [ ]:
pd.DataFrame(embeddings_bert).to_csv('embeddings_bert.csv', index=False)

## Curseword Detection

Refer to https://github.com/vzhou842/profanity-check to use as an indicator for curseword

In [ ]:
featured_df['curseWord'] =  featured_df['clean_text'].apply(lambda x: predict([x])[0])

## Unique, Repated words

In [95]:
clean_df = processed_df.iloc[:,1:8]
#clean_df['text_tokenized'] = clean_df['text_tokenized'].apply(lambda x: [c for c in x if c not in string.punctuation])

In [152]:
#Without removing stopwords but punctuation

def freq(text):
    text = [c for c in text if c not in string.punctuation]
    if len(text) == 0:
        return [0,0,0,0,0,0,0,0]
    try:
        #get frequency vectore
        vec = CountVectorizer(ngram_range=(1, 1), stop_words=None)
        bow = vec.fit_transform([' '.join(text)])
    except:
        print(text)
        #get frequency vectore
        vec = CountVectorizer(ngram_range=(1, 1), stop_words=None)
        bow = vec.fit_transform([''.join(text)])
        print(bow)
    finally:          
        sum_of_words = bow.sum(axis=0)
        unigrams_freq = [(word, sum_of_words[0, idx]) for word, idx in vec.vocabulary_.items()]

        #how many words
        words = len(unigrams_freq)
        unigrams_freq = pd.DataFrame(unigrams_freq,columns=['word','freq'])

        # length of text
        total_length = len(''.join(unigrams_freq.word))

        #length of repeated text
        repeated_length = len(''.join(unigrams_freq['word'][unigrams_freq.freq>1]))
        #length of repeated text
        unique_length = total_length-repeated_length

        #how many repeated words
        repeated_words = unigrams_freq[unigrams_freq.freq>1]
        repeated_words_length = len(unigrams_freq[unigrams_freq.freq>1])

        #how many unqiue words
        unique_words_length = words - repeated_words_length


        repeated_words_vs_length = repeated_length / total_length
        repeated_words_vs_words = repeated_words_length / words
        unique_words_vs_length = unique_length / total_length
        unique_words_vs_words = unique_words_length / words
        return [words, total_length, repeated_words_length, repeated_words_vs_length, repeated_words_vs_words,
                 unique_words_length,unique_words_vs_length, unique_words_vs_words]



In [153]:
clean_df['words'],clean_df['total_length'],clean_df['repeated_words_length'],\
clean_df['repeated_words_vs_length'],clean_df['repeated_words_vs_words'],\
clean_df['unique_words_length'],clean_df['unique_words_vs_length'],\
clean_df['unique_words_vs_words'] = zip(*clean_df['text_tokenized'].apply(
    lambda comment: freq(comment)))

['n', 'i', 'g', 'g', 'e', 'r', 'f', 'a', 'g', 'g', 'o', 't']
  (0, 0)	1
['p', 'e', 'n', 'i', 's']
  (0, 0)	1
['p', 'e', 'n', 'i', 's']
  (0, 0)	1


In [154]:
clean_df.columns

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate',
       'text_tokenized', 'words', 'total_length', 'repeated_words_vs_length',
       'repeated_words_vs_words', 'unique_words_vs_length',
       'unique_words_vs_words', 'repeated_words_length',
       'unique_words_length'],
      dtype='object')

In [156]:
LABELS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
features = ('words', 'total_length', 'repeated_words_vs_length',
       'repeated_words_vs_words', 'unique_words_vs_length',
       'unique_words_vs_words', 'repeated_words_length',
       'unique_words_length'
           )
clean_df['none'] = 1 - clean_df[LABELS].max(axis=1)

In [157]:
columns = LABELS + ['none']

rows = [{c:clean_df[f].corr(clean_df[c]) for c in columns} for f in features]
train_correlations_unique = pd.DataFrame(rows, index=features)
train_correlations_unique

,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
words,-0.099039,-0.050574,-0.083467,-0.020974,-0.083366,-0.033696,0.094470
total_length,-0.098959,-0.047987,-0.083478,-0.022275,-0.082849,-0.031375,0.095204
repeated_words_vs_length,-0.016289,0.048949,-0.011058,0.012372,-0.008599,0.005186,0.015832
repeated_words_vs_words,-0.031051,0.034609,-0.026098,0.008308,-0.021725,-0.001735,0.029628
unique_words_vs_length,0.016496,-0.048752,0.011210,-0.012300,0.008750,-0.005101,-0.016047
unique_words_vs_words,0.031218,-0.034470,0.026212,-0.008254,0.021842,0.001798,-0.029806
repeated_words_length,-0.085243,-0.035968,-0.070000,-0.016286,-0.070045,-0.027245,0.081918
unique_words_length,-0.102455,-0.055957,-0.087231,-0.022547,-0.087064,-0.035704,0.097437


## Nouns, Verbs, Adjectives

In [5]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [8]:
from nltk.tag import pos_tag

pos_analysis_df = train_df.copy()

def tag_part_of_speech(text):
    text_splited = text.split(' ')
    text_splited = [''.join(c for c in s if c not in string.punctuation) for s in text_splited]
    text_splited = [s for s in text_splited if s]
    pos_list = pos_tag(text_splited)
    noun_count = len([w for w in pos_list if w[1] in ('NN','NNP','NNPS','NNS')])
    adjective_count = len([w for w in pos_list if w[1] in ('JJ','JJR','JJS')])
    verb_count = len([w for w in pos_list if w[1] in ('VB','VBD','VBG','VBN','VBP','VBZ')])
    return[noun_count, adjective_count, verb_count]


pos_analysis_df['nouns'], pos_analysis_df['adjectives'], pos_analysis_df['verbs'] = zip(*pos_analysis_df['comment_text'].apply(
    lambda comment: tag_part_of_speech(comment)))


In [11]:
pos_analysis_df['total_length'] = pos_analysis_df['comment_text'].apply(len)

pos_analysis_df['words'] = pos_analysis_df['comment_text'].apply(lambda comment: len(comment.split()))

In [12]:
pos_analysis_df['nouns_vs_length'] = pos_analysis_df['nouns'] / pos_analysis_df['total_length']
pos_analysis_df['adjectives_vs_length'] = pos_analysis_df['adjectives'] / pos_analysis_df['total_length']
pos_analysis_df['verbs_vs_length'] = pos_analysis_df['verbs'] / pos_analysis_df['total_length']
pos_analysis_df['nouns_vs_words'] = pos_analysis_df['nouns'] / pos_analysis_df['words']
pos_analysis_df['adjectives_vs_words'] = pos_analysis_df['adjectives'] / pos_analysis_df['words']
pos_analysis_df['verbs_vs_words'] = pos_analysis_df['verbs'] / pos_analysis_df['words']

In [13]:
pos_analysis_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,nouns,adjectives,verbs,total_length,words,nouns_vs_length,adjectives_vs_length,verbs_vs_length,nouns_vs_words,adjectives_vs_words,verbs_vs_words
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,17,2,8,264,43,0.064394,0.007576,0.030303,0.395349,0.046512,0.186047
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,7,0,3,112,17,0.062500,0.000000,0.026786,0.411765,0.000000,0.176471
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,11,2,7,233,42,0.047210,0.008584,0.030043,0.261905,0.047619,0.166667
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,24,6,23,622,113,0.038585,0.009646,0.036977,0.212389,0.053097,0.203540
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,4,0,3,67,13,0.059701,0.000000,0.044776,0.307692,0.000000,0.230769


In [15]:
LABELS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

features = (
            'nouns', 'nouns_vs_words', 'nouns_vs_length', 
            'adjectives', 'adjectives_vs_words', 'adjectives_vs_length',
            'verbs', 'verbs_vs_words', 'verbs_vs_length',
           )
pos_analysis_df['none'] = 1 - pos_analysis_df[LABELS].max(axis=1)

In [17]:
columns = LABELS + ['none']

rows = [{c:pos_analysis_df[f].corr(pos_analysis_df[c]) for c in columns} for f in features]
train_correlations = pd.DataFrame(rows, index=features)
train_correlations

,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
nouns,-0.000861,0.068455,0.001967,0.003365,-0.001149,0.009624,-0.000103
nouns_vs_words,0.125599,0.113773,0.121616,0.021363,0.110377,0.066107,-0.121250
nouns_vs_length,0.178798,0.140118,0.168074,0.036472,0.153487,0.085114,-0.174803
adjectives,-0.045447,0.004472,-0.036740,-0.015309,-0.032686,-0.000146,0.042934
adjectives_vs_words,0.028471,0.005824,0.016104,-0.011183,0.030969,0.034752,-0.029019
adjectives_vs_length,0.047081,0.012349,0.032990,-0.005735,0.046271,0.043044,-0.047881
verbs,-0.061767,-0.014051,-0.051171,-0.003157,-0.051491,-0.021670,0.057725
verbs_vs_words,-0.004188,-0.032360,-0.011213,0.007587,-0.008274,-0.013781,0.002575
verbs_vs_length,0.024287,-0.019703,0.013053,0.020133,0.011791,-0.005127,-0.026003
